<a href="https://colab.research.google.com/github/MatinAminSabouri/Food_RAG_System/blob/main/Food_Rag_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q transformers datasets torch torchvision matplotlib wikipedia accelerate bitsandbytes

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 117.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 47.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.9/72.9 MB 10.3 MB/s eta 0:00:00


In [2]:
from PIL import Image
import requests
import torch
from transformers import CLIPProcessor, CLIPModel

# Load CLIP model and processor
model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32")
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


model.safetensors:   0%|          | 0.00/605M [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

In [3]:
from google.colab import files
uploaded = files.upload()  # opens a file picker

image = Image.open("sushi.jpg")
image.show()

# Define possible food labels (just for my MVP)
food_labels = ["Spaghetti Carbonara", "Hamburger", "Sushi", "Caesar Salad", "Chicken Tikka",
               "Pad Thai", "Pho", "Ramen", "Pizza Margherita", "Tacos"]


Saving sushi.jpg to sushi.jpg


In [4]:
# Prepare inputs
inputs = processor(text=food_labels, images=image, return_tensors="pt", padding=True)


In [5]:
# Predict
with torch.no_grad():
    outputs = model(**inputs)
    logits_per_image = outputs.logits_per_image
    probs = logits_per_image.softmax(dim=1)

# Get prediction
predicted_index = torch.argmax(probs).item()
predicted_label = food_labels[predicted_index]
#print(f"🍽️ Predicted dish: {predicted_label}")


In [7]:
import os

os.environ["SPOONACULAR_API_KEY"] = "apikey"


In [8]:
import os
import requests

API_KEY = os.environ.get("SPOONACULAR_API_KEY")

if not API_KEY:
    raise ValueError("API key not found. Please set the SPOONACULAR_API_KEY environment variable.")


In [9]:
def get_recipe(dish_name):
    search_url = "https://api.spoonacular.com/recipes/complexSearch"
    params = {
        "query": dish_name,
        "number": 1,
        "apiKey": API_KEY
    }

    #Search food
    response = requests.get(search_url, params=params)
    data = response.json()

    if not data["results"]:
        return "No Result!"

    recipe_id = data["results"][0]["id"]
    title = data["results"][0]["title"]

    info_url = f"https://api.spoonacular.com/recipes/{recipe_id}/information"
    info_params = {
        "includeNutrition": False,
        "apiKey": API_KEY
    }
    info_response = requests.get(info_url, params=info_params)
    info_data = info_response.json()

    ingredients = [ing["original"] for ing in info_data["extendedIngredients"]]
    instructions = info_data.get("instructions", "No instructions provided.")

    return {
        "title": title,
        "ingredients": ingredients,
        "instructions": instructions
    }


In [10]:
print(f"🍽️ Predicted dish: {predicted_label}")

recipe = get_recipe(predicted_label)

# print("\n🍽️ Title:", recipe["title"])
# print("\n🧂 Ingredients:")
# for i, ing in enumerate(recipe["ingredients"], 1):
#     print(f"{i}. {ing}")

#print("\n👨‍🍳 Instructions:\n", recipe["instructions"])


🍽️ Predicted dish: Sushi


In [11]:
import wikipedia

wikipedia.set_lang("en")


In [12]:
def get_wikipedia_summary(food_name):
    try:
        summary = wikipedia.summary(food_name, sentences=3)
        return summary
    except wikipedia.exceptions.DisambiguationError as e:
        return f"Ambiguous term. Try: {e.options[:3]}"
    except wikipedia.exceptions.PageError:
        return "No Wikipedia page found."


In [13]:
print(f"🍽️ Predicted dish: {predicted_label}")

recipe = get_recipe(predicted_label)

print("\n🍽️ Title:", recipe["title"])
print("\n🧂 Ingredients:")
for i, ing in enumerate(recipe["ingredients"], 1):
    print(f"{i}. {ing}")

print("\n👨‍🍳 Instructions:\n", recipe["instructions"])

history = get_wikipedia_summary(predicted_label)
print("\n📚 History:")
print(history)


🍽️ Predicted dish: Sushi

🍽️ Title: Japanese Sushi

🧂 Ingredients:
1. Cooked octopus
2. Cooked prawns
3. Raw tuna
4. Salmon
5. Salmon caviar
6. Japanese sticky rice
7. Lava seaweed
8. Wasabi
9. Asparagus
10. Shiitake mushrooms

👨‍🍳 Instructions:
 Recipe one: Pour cooked rice into a tray. Finely slice the salmon and the tuna.
Put the seaweed onto a sushi mat and press rice down onto it.
Slice the mushrooms and asparagus pieces and lay along the rice. Add the wasabi. Roll the sushi and cut into slices. Serve with salmon caviar, wasabi and soy.
Recipe two: Squeeze the rice into balls and press the prawns, tuna or salmon into the top.

📚 History:
Su Shi (simplified Chinese: 苏轼; traditional Chinese: 蘇軾; pinyin: Sū Shì; 8 January 1037 – 24 August 1101), courtesy name Zizhan (子瞻), art name Dongpo (東坡), was a Chinese poet, essayist, calligrapher, painter, scholar-official, literatus, artist, pharmacologist, and gastronome who lived during the Song dynasty. A major personality of the Song era, 

In [15]:
from transformers import AutoTokenizer, AutoModelForCausalLM
import torch

model_id = "microsoft/phi-2"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype=torch.float16,
    trust_remote_code=True
)


tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

added_tokens.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

In [16]:
def generate_text(dish, recipe, history):
    prompt = f"""You are a food assistant. Your job is to explain a dish.

Dish: {dish}

Here is the recipe:
Ingredients:
{chr(10).join(recipe['ingredients'])}

Instructions:
{recipe['instructions']}

Here is some background info:
{history}

Please write a clear, friendly explanation of what this food is, how it's made, and its cultural background.
"""
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)
    output = model.generate(**inputs, max_new_tokens=250, do_sample=True, top_p=0.9)
    return tokenizer.decode(output[0], skip_special_tokens=True)


In [17]:
result = generate_text(predicted_label, recipe, history)
print(result)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a food assistant. Your job is to explain a dish.

Dish: Sushi

Here is the recipe:
Ingredients:
Cooked octopus
Cooked prawns
Raw tuna
Salmon
Salmon caviar
Japanese sticky rice
Lava seaweed
Wasabi
Asparagus
Shiitake mushrooms

Instructions:
Recipe one: Pour cooked rice into a tray. Finely slice the salmon and the tuna.
Put the seaweed onto a sushi mat and press rice down onto it.
Slice the mushrooms and asparagus pieces and lay along the rice. Add the wasabi. Roll the sushi and cut into slices. Serve with salmon caviar, wasabi and soy.
Recipe two: Squeeze the rice into balls and press the prawns, tuna or salmon into the top.

Here is some background info:
Su Shi (simplified Chinese: 苏轼; traditional Chinese: 蘇軾; pinyin: Sū Shì; 8 January 1037 – 24 August 1101), courtesy name Zizhan (子瞻), art name Dongpo (東坡), was a Chinese poet, essayist, calligrapher, painter, scholar-official, literatus, artist, pharmacologist, and gastronome who lived during the Song dynasty. A major personali